Алгоритм решения МППР итерацией по стратегиям состоит из 2х шагов: оценка стратегии и её улучшение. Первоначальную стратегию выбираем произвольно. Затем итеративно оцениваем её до сходимости с помощью уравнения мат ожидания Беллмана: 

$V(s) = \sum_{s'} T(s,a,s') \left[ R(s,a) + \gamma  V(s') \right]$

Уравнение похоже на то, что мы использовали ранее, просто теперь вычисление вознаграждения осуществляется как мат ожидание по всем состояниям для действия $a = \pi(s)$, которое возвращает стратегия.

На основе полученной фукции ценности $V(s)$ улучшаем стратегию с помощь уравнения оптимальности

$\pi(s) = argmax_a \sum_{s'}T(s,a,s')\left[R(s,a,s') + \gamma V(s')\right]$

Шаги повторяются, пока стратегия не сойдется. Стратегия и функция ценности, соответствующие неподвижной точке являются оптимальными.

In [1]:
import torch
import gym

env = gym.make('FrozenLake-v0')

gamma = 0.99

threshold = 0.0001

Обращу внимание на то, что данный алгоритм предполагает два этапа итерации до сходимости: внутренний - при оценке текущей стратегии, и внешний - при улучшении стратегии.

In [2]:
def policy_evaluation(env, policy, gamma, threshold):
    """
    Perform policy evaluation
    @param env: OpenAI Gym environment
    @param policy: policy matrix containing actions and their probability in each state
    @param gamma: discount factor
    @param threshold: the evaluation will stop once values for all states are less than the threshold
    @return: values of the given policy
    """
    n_state = policy.shape[0]
    V = torch.zeros(n_state)
    while True:
        V_temp = torch.zeros(n_state)
        for state in range(n_state):
            action = policy[state].item()
            for trans_prob, new_state, reward, _ in env.env.P[state][action]:
                V_temp[state] += trans_prob * (reward + gamma * V[new_state])
        max_delta = torch.max(torch.abs(V - V_temp))
        V = V_temp.clone()
        if max_delta <= threshold:
            break
    return V

In [3]:
def policy_improvement(env, V, gamma):
    """
    Obtain an improved policy based on the values
    @param env: OpenAI Gym environment
    @param V: policy values
    @param gamma: discount factor
    @return: the policy
    """
    n_state = env.observation_space.n
    n_action = env.action_space.n
    policy = torch.zeros(n_state)
    for state in range(n_state):
        v_actions = torch.zeros(n_action)
        for action in range(n_action):
            for trans_prob, new_state, reward, _ in env.env.P[state][action]:
                v_actions[action] += trans_prob * (reward + gamma * V[new_state])
        policy[state] = torch.argmax(v_actions)
    return policy


In [4]:
def policy_iteration(env, gamma, threshold):
    """
    Solve a given environment with policy iteration algorithm
    @param env: OpenAI Gym environment
    @param gamma: discount factor
    @param threshold: the evaluation will stop once values for all states are less than the threshold
    @return: optimal values and the optimal policy for the given environment
    """
    n_state = env.observation_space.n
    n_action = env.action_space.n
    policy = torch.randint(high=n_action, size=(n_state,)).float()
    while True:
        V = policy_evaluation(env, policy, gamma, threshold)
        policy_improved = policy_improvement(env, V, gamma)
        if torch.equal(policy_improved, policy):
            return V, policy_improved
        policy = policy_improved

In [5]:
V_optimal, optimal_policy = policy_iteration(env, gamma, threshold)
print('Optimal values:\n{}'.format(V_optimal))
print('Optimal policy:\n{}'.format(optimal_policy))

Optimal values:
tensor([0.5404, 0.4966, 0.4681, 0.4541, 0.5569, 0.0000, 0.3572, 0.0000, 0.5905,
        0.6421, 0.6144, 0.0000, 0.0000, 0.7410, 0.8625, 0.0000])
Optimal policy:
tensor([0., 3., 3., 3., 0., 0., 0., 0., 3., 1., 0., 0., 0., 2., 1., 0.])


При выборе схемы поиска оптимальной стратегии для конкретной задачи будем пользоваться следующими соображениями: если действий немного, то итерация по ценности будет работать хорошо, для большого количества действий скорость поиска стратегии будет в среднем выше у итерации по стратегиям. Если есть неплохая стартовая стратегия, полученная из каких-то соображений, то можно начинать прямо с неё и итерировать по стратегиям.

К настоящему моменту, я надеюсь, что вам удалось разобраться со следующими понятиями: 
функция ценности, стратегия, оценка стратегии, алгоритмы поиска оптимальной стратегии итерацией итерацией по ценности и итерацией по стратегиям. Это важные инструменты, которые понадобятся нам в будущем, чтобы строить сложные и эффективные алгоритмы.